# Fees Sankey Diagram Prototype

This notebook demonstrates how to generate and visualize Superchain fee flows using a Sankey diagram.

We'll focus on **Base** with **28 days** of data as our test case.

## Overview

The Sankey diagram shows the hierarchical breakdown of fees:
- **Level 1 (Main categories):** Chain, MEV, Stablecoin, App fees (must sum to 100%)
- **Level 2 (Sub-breakdowns):** Revenue shares and components within each category

## Revenue Flow Structure

**Chain fees:** Revenue share to Optimism + Revenue to Chain + Gas costs + Remaining  
**MEV fees:** Revenue share to App + Remaining  
**Stablecoin fees:** Revenue share to App + Remaining  
**App fees:** Revenue to App + Remaining  


In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import iplot
import plotly.io as pio

# Set up plotly for notebook display
pio.renderers.default = "notebook"

# Import our fees sankey function
from op_analytics.transforms.fees_sankey.generate_sankey_fees_dataset import execute_pull

print("✅ Imports successful!")


✅ Imports successful!


## Generate Sankey Data

Let's generate the fee flow data using our transform function with dry-run mode to avoid writing to databases.


In [2]:
# Generate the fee flow data (28 days lookback, dry-run mode)
result = execute_pull(days=28, dry_run=True)

print("Execution Summary:")
for key, value in result.items():
    print(f"  {key}: {value}")

# Get the generated DataFrame
df = result['dataframe']
print(f"\n📊 Generated {len(df)} fee flow edges")
print(f"📈 Total fee value: ${df['value'].sum():,.2f}")

# Show sample of the data
print("\n🔍 Sample data:")
df.head(10)


2025-07-15 16:43:23 [info     ] Starting Sankey fees dataset generation days=28 dry_run=True filename=generate_sankey_fees_dataset.py lineno=411 pipeline_step=fees_sankey process=46820
2025-07-15 16:43:25 [info     ] Querying source data           filename=generate_sankey_fees_dataset.py lineno=417 pipeline_step=fees_sankey process=46820 table=oplabs-tools-data.materialized_tables.daily_superchain_health_mv
2025-07-15 16:43:26 [info     ] Retrieved chains with fee data chains_count=66 filename=generate_sankey_fees_dataset.py lineno=429 max_rss=192.1 pipeline_step=fees_sankey process=46820
2025-07-15 16:43:26 [info     ] Fee data summary               filename=generate_sankey_fees_dataset.py lineno=435 max_fees_usd=np.float64(676014.7496697985) min_fees_usd=np.float64(0.003217326523783194) pipeline_step=fees_sankey process=46820
2025-07-15 16:43:26 [info     ] Processing fee flows           filename=generate_sankey_fees_dataset.py lineno=446 pipeline_step=fees_sankey process=46820
2025-

KeyError: 'dataframe'

## Filter for Base

Let's focus on Base data for our Sankey visualization test case.


In [ ]:
# Filter for Base
base_df = df[df['chain_set'] == 'Base'].copy()

print(f"📍 Base fee flow edges: {len(base_df)}")
print(f"💰 Base total fees: ${base_df['value'].sum():,.2f}")

# Show breakdown by level
level1_df = base_df[base_df['source'] == 'Total Fees']
level2_df = base_df[base_df['source'] != 'Total Fees']

print(f"\n📊 Level 1 edges (main categories): {len(level1_df)}")
print(f"📊 Level 2 edges (sub-breakdowns): {len(level2_df)}")

print(f"\n✅ Level 1 percentage check: {level1_df['pct_of_total_fees_usd'].sum():.1f}% (should be 100%)")

print("\n🔍 Base data:")
base_df.sort_values('value', ascending=False)


## Create Sankey Diagram

Now let's create a beautiful Sankey diagram using Plotly. We'll need to:
1. Build node lists (all unique sources and destinations)
2. Create links with proper indices
3. Apply colors and formatting


In [ ]:
def create_sankey_diagram(df, title="Superchain Fee Flows"):
    """Create a Plotly Sankey diagram from fee flow data."""
    
    # Filter out zero-value flows for cleaner visualization
    df_filtered = df[df['value'] > 0].copy()
    
    # Get all unique nodes (sources and destinations)
    all_sources = df_filtered['source'].unique()
    all_destinations = df_filtered['destination'].unique()
    all_nodes = list(set(list(all_sources) + list(all_destinations)))
    
    # Create node index mapping
    node_indices = {node: i for i, node in enumerate(all_nodes)}
    
    # Create source and target indices for links
    source_indices = [node_indices[source] for source in df_filtered['source']]
    target_indices = [node_indices[dest] for dest in df_filtered['destination']]
    values = df_filtered['value'].tolist()
    
    # Define colors for different node types
    node_colors = []
    for node in all_nodes:
        if node == 'Total Fees':
            node_colors.append('#1f77b4')  # Blue for total
        elif 'fees' in node.lower():
            node_colors.append('#ff7f0e')  # Orange for fee categories
        elif 'revenue' in node.lower() or 'optimism' in node.lower():
            node_colors.append('#2ca02c')  # Green for revenue
        elif 'remaining' in node.lower():
            node_colors.append('#d62728')  # Red for remaining
        elif 'gas' in node.lower():
            node_colors.append('#9467bd')  # Purple for gas costs
        else:
            node_colors.append('#8c564b')  # Brown for others
    
    # Create labels with values for better readability
    node_labels = []
    for node in all_nodes:
        # Calculate total inflow for this node
        inflow = df_filtered[df_filtered['destination'] == node]['value'].sum()
        if inflow > 0:
            node_labels.append(f"{node}<br>${inflow:,.0f}")
        else:
            node_labels.append(node)
    
    # Create the Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_labels,
            color=node_colors
        ),
        link=dict(
            source=source_indices,
            target=target_indices,
            value=values,
            color=['rgba(255,127,14,0.4)' for _ in values]  # Semi-transparent orange
        )
    )])\n    
    fig.update_layout(
        title_text=f"{title}<br><sub>28-day lookback • Values in USD</sub>",
        font_size=10,
        width=1000,
        height=600
    )\n    
    return fig

# Create and display the Sankey diagram
sankey_fig = create_sankey_diagram(base_df, "Base Fee Flows")
sankey_fig.show()


In [ ]:
# Detailed analysis of Base fee flows
print("🔍 BASE FEE FLOW ANALYSIS")
print("=" * 50)

total_fees = base_df['value'].sum()
print(f"💰 Total Fee Volume: ${total_fees:,.2f}")

print(f"\n📊 LEVEL 1 BREAKDOWN (Main Categories):")
for _, row in level1_df.sort_values('value', ascending=False).iterrows():
    print(f"  • {row['destination']:20} ${row['value']:>10,.0f} ({row['pct_of_total_fees_usd']:>5.1f}%)")

print(f"\n🔧 LEVEL 2 BREAKDOWN (Sub-components):")
for _, row in level2_df.sort_values('value', ascending=False).iterrows():
    print(f"  • {row['source']} → {row['destination']}")
    print(f"    ${row['value']:,.0f}")

print(f"\n✅ VALIDATION:")
print(f"  • Level 1 percentages sum to: {level1_df['pct_of_total_fees_usd'].sum():.1f}%")
print(f"  • Total edges generated: {len(base_df)}")
print(f"  • Zero-value edges: {len(base_df[base_df['value'] == 0])}")

print(f"\n🎯 READY FOR VISUALIZATION!")
print(f"  • Data structure validated ✅")
print(f"  • Percentages correctly calculated ✅")
print(f"  • Sankey diagram rendered above ✅")


# Fees Sankey Transform Prototype

This notebook is for prototyping and testing the fees sankey transform that generates datasets for Sankey diagram visualization of Superchain fee flows.

## Purpose
- Test fee flow logic
- Validate output structure
- Prototype new features
- Backfill historical data if needed


In [ ]:
import sys
import os
sys.path.append('../../../src')

import pandas as pd
import polars as pl
from op_analytics.coreutils.logger import structlog
from op_analytics.transforms.fees_sankey.generate_sankey_fees_dataset import (
    get_source_data, 
    process_fee_flows, 
    validate_output
)

log = structlog.get_logger()


## Test Configuration


In [ ]:
# Configuration
DAYS = 30  # Look back 30 days for testing
DRY_RUN = True  # Don't write to databases during prototyping

print(f"Configuration: {DAYS} days lookback, dry_run={DRY_RUN}")


## Run Transform

Execute the fees sankey transform with dry run for testing:


In [ ]:
# Run the transform execute_pull function for testing
from op_analytics.transforms.fees_sankey.generate_sankey_fees_dataset import execute_pull

# This will run the full pipeline in dry-run mode
try:
    result = execute_pull(days=DAYS, dry_run=DRY_RUN)
    print("✅ Transform completed successfully!")
    print(f"Summary: {result}")
except Exception as e:
    print(f"❌ Transform failed: {e}")
    import traceback
    traceback.print_exc()
